In [3]:
#@markdown #掛載Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.9 MB/s eta 0:00:00


In [2]:
"""
技術指標 Autoencoder 壓縮訓練腳本
將同類技術指標壓縮為單一數值
"""

import sys
import io

# 設定輸出編碼為 UTF-8（解決 Windows 控制台編碼問題）
if sys.platform == 'win32':
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8', errors='replace')
    sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8', errors='replace')

import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import warnings
from datetime import datetime
import json
import random

# 在導入 TensorFlow 之前設定環境變數（幫助 TensorFlow 找到 CUDA）
# 這可以幫助 TensorFlow 在 Windows 上找到 CUDA 庫
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # 減少 TensorFlow 日誌輸出
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # 關閉 oneDNN 選項

# 深度學習相關
import tensorflow as tf
# TensorFlow 2.10+ 中，keras 是獨立包，需要使用 keras.src
import keras
from keras.src import layers, models, callbacks
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

# 圖表和輸出
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  # 使用非交互式後端
import openpyxl
from openpyxl.drawing.image import Image
import io

warnings.filterwarnings('ignore')

In [4]:
# ==================== 可調整參數 ====================
# 技術指標群組定義
INDICATOR_GROUPS = {
    "STOCH": ["STOCH_K_14", "STOCH_D_14"],
    "STOCHF": ["STOCHF_K_14", "STOCHF_D_14"],
    "STOCHRSI": ["STOCHRSI_K_14", "STOCHRSI_D_14"],
    "MACD": ["MACD_12_26", "MACD_signal_12_26", "MACD_hist_12_26"],
    "BBANDS": ["BBANDS_upper_20", "BBANDS_middle_20", "BBANDS_lower_20"],
    "ADX_DMI": ["ADX_14", "ADXR_14", "PDI_14", "MDI_14", "DX_14"],
    "AROON": ["AROON_Down_14", "AROON_Up_14", "AROONOSC_14"]
}

# 資料路徑

In [8]:
DATA_DIR = "/content/drive/MyDrive/工作區/論文/論文code/1106/technical_indicators_data_extracted"
OUTPUT_DIR = "/content/drive/MyDrive/工作區/論文/論文code/1106/output2"

In [9]:
import os

def list_top_n_files(directory, n=5):
    """Lists the first n files in a given directory."""
    if not os.path.isdir(directory):
        print(f"Error: Directory not found: {directory}")
        return

    print(f"\nListing first {n} files in: {directory}")
    try:
        files = os.listdir(directory)
        # Sort files to get a consistent order
        files.sort()
        for i, file in enumerate(files[:n]):
            print(f"- {file}")
        if len(files) > n:
            print(f"... and {len(files) - n} more files")
    except Exception as e:
        print(f"Error listing files in {directory}: {e}")

# List files in DATA_DIR
list_top_n_files(DATA_DIR)

# List files in OUTPUT_DIR
list_top_n_files(OUTPUT_DIR)


Listing first 5 files in: /content/drive/MyDrive/工作區/論文/論文code/1106/technical_indicators_data_extracted
- TX20110103_1K_qlib_indicators_complete.csv
- TX20110104_1K_qlib_indicators_complete.csv
- TX20110105_1K_qlib_indicators_complete.csv
- TX20110106_1K_qlib_indicators_complete.csv
- TX20110107_1K_qlib_indicators_complete.csv
... and 2719 more files

Listing first 5 files in: /content/drive/MyDrive/工作區/論文/論文code/1106/output2


In [22]:
# ======== 全域參數（避免重複定義） ========
VAL_SPLIT = 0.2  # 只用於 Train 內的驗證切分

ENCODER_DIMS = [256, 128]
DECODER_DIMS = [128, 256]

# Dropout 刻度（0~0.4 常見甜區）
DROPOUT_RATES = [0.0, 0.05, 0.1, 0.2, 0.3, 0.4]

# Batch size（依 GPU 記憶體調整）
BATCH_SIZE = 32786

# ==== 貝葉斯優化參數 ====
BAYESIAN_N_CALLS = 24

# 訓練參數
EARLY_STOPPING_PATIENCE = 16
MAX_EPOCHS = 300
RANDOM_SEED = 42

# 搜索階段的早停耐心（較短，加快搜索）
SEARCH_EARLY_STOPPING_PATIENCE = 8

# 優化選項
SKIP_FINAL_TRAINING = True  # True：跳過最終訓練，直接用搜尋階段最佳模型

# ==================== 超參數搜尋空間 ====================
# 固定瓶頸層大小（全部壓成 1 維）
FIXED_BOTTLENECK = 1

# 改為連續取值（log-uniform 分佈）
from skopt.space import Real, Integer
LEARNING_RATE_SPACE = Real(3e-4, 2e-3, prior='log-uniform', name='learning_rate')

In [24]:
# @title
# ==================== 設定隨機種子 ====================
def set_random_seeds(seed):
    """設定所有隨機種子以確保可重現性"""
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_random_seeds(RANDOM_SEED)

# ==================== 資料載入 ====================
def load_all_data(data_dir):
    """載入所有CSV檔案並合併"""
    print("=" * 60)
    print("[LOAD] 載入資料...")

    csv_files = glob.glob(os.path.join(data_dir, "TX*_1K_qlib_indicators_complete.csv"))
    csv_files.sort()  # 按檔名排序以確保時間順序

    print(f"找到 {len(csv_files)} 個CSV檔案")

    all_data = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df['datetime'] = pd.to_datetime(df['datetime'])
            all_data.append(df)
        except Exception as e:
            print(f"[WARN] 讀取檔案失敗: {os.path.basename(file)} - {e}")

    if not all_data:
        raise ValueError("[ERROR] 沒有成功載入任何資料！")

    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime').reset_index(drop=True)

    print(f"[OK] 成功載入資料，總共 {len(combined_df):,} 筆記錄")
    print(f"[INFO] 時間範圍: {combined_df['datetime'].min()} 至 {combined_df['datetime'].max()}")

    return combined_df
df = load_all_data(DATA_DIR)

[LOAD] 載入資料...
找到 2723 個CSV檔案
[OK] 成功載入資料，總共 819,623 筆記錄
[INFO] 時間範圍: 2011-01-03 08:45:00 至 2023-12-22 13:45:00


In [ ]:
# @title
# ==================== 資料切分（3 年視窗：前 2 年訓練、後 1 年測試） ====================
def split_train_val_by_date(df_in, val_split=0.2):
    """只在訓練資料內做 Train/Val 切分（以交易日為單位）"""
    if not pd.api.types.is_datetime64_any_dtype(df_in['datetime']):
        df_in['datetime'] = pd.to_datetime(df_in['datetime'])

    unique_dates = sorted(df_in['datetime'].dt.date.unique())
    if len(unique_dates) < 2:
        raise ValueError("[ERROR] 訓練資料日期不足，無法切分 Train/Val")

    n_days = len(unique_dates)
    n_val_days = max(1, int(n_days * val_split))
    n_train_days = n_days - n_val_days

    val_start_date = unique_dates[n_train_days]

    train_mask = df_in['datetime'].dt.date < val_start_date
    val_mask = df_in['datetime'].dt.date >= val_start_date

    train_df = df_in[train_mask].copy()
    val_df = df_in[val_mask].copy()

    print("[SPLIT] Train/Val by trading day")
    print(f"  訓練集天數: {n_train_days}")
    print(f"  驗證集天數: {n_val_days}")
    print(f"  Cutoff (Train|Val): {val_start_date}")
    print(f"  訓練集: {len(train_df):,} 筆 | 時間: {train_df['datetime'].min()} 至 {train_df['datetime'].max()}")
    print(f"  驗證集: {len(val_df):,} 筆 | 時間: {val_df['datetime'].min()} 至 {val_df['datetime'].max()}")

    return train_df, val_df


def build_rolling_windows_by_year(df_in, window_years=3):
    """建立可用的 3 年滾動視窗（若年份缺失則跳過該視窗）"""
    if not pd.api.types.is_datetime64_any_dtype(df_in['datetime']):
        df_in['datetime'] = pd.to_datetime(df_in['datetime'])

    data_years = sorted(df_in['datetime'].dt.year.unique())
    if len(data_years) < window_years:
        raise ValueError("[ERROR] 資料年份不足 3 年，無法進行 3 年滾動視窗。")

    full_years = list(range(data_years[0], data_years[-1] + 1))
    windows = []
    for i in range(len(full_years) - window_years + 1):
        window = full_years[i:i + window_years]
        if all(y in data_years for y in window):
            windows.append(window)
        else:
            missing = [y for y in window if y not in data_years]
            print(f"[WARN] 跳過視窗 {window[0]}-{window[-1]}，缺少年份: {missing}")

    if not windows:
        raise ValueError("[ERROR] 沒有可用的完整 3 年視窗。")

    return windows

# ==================== 資料準備 ====================
def prepare_indicator_data(df, indicator_cols):
    """準備指定指標的資料"""
    missing_cols = [col for col in indicator_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"[ERROR] 缺少欄位: {missing_cols}")

    data = df[indicator_cols].values
    data = np.nan_to_num(data, nan=0.0, posinf=1e6, neginf=-1e6)
    return data

# ==================== Autoencoder 模型 ====================
def build_autoencoder(input_dim, bottleneck_size, dropout_rate=0.0):
    """建立 Autoencoder 模型

    架構: Input → 256 → 128 → bottleneck → 128 → 256 → Output
    """
    input_layer = layers.Input(shape=(input_dim,), name='input')

    # 編碼器
    x = layers.Dense(ENCODER_DIMS[0], activation='relu', name='encoder_1')(input_layer)
    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ENCODER_DIMS[1], activation='relu', name='encoder_2')(x)
    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)
    bottleneck = layers.Dense(bottleneck_size, activation='relu', name='bottleneck')(x)

    # 解碼器
    x = layers.Dense(DECODER_DIMS[0], activation='relu', name='decoder_1')(bottleneck)
    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(DECODER_DIMS[1], activation='relu', name='decoder_2')(x)
    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)
    output = layers.Dense(input_dim, activation='linear', name='output')(x)

    model = models.Model(inputs=input_layer, outputs=output, name='autoencoder')
    return model

# ==================== 訓練函數 ====================
def train_autoencoder(X_train, X_val, bottleneck_size, lr, dropout_rate, batch_size,
                     max_epochs=200, patience=12, group_name="", show_progress=False):
    """訓練單一 Autoencoder"""
    input_dim = X_train.shape[1]

    model = build_autoencoder(input_dim, bottleneck_size, dropout_rate)

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True,
        verbose=0
    )

    history_callback = callbacks.History()

    class ProgressCallback(callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            if show_progress and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{max_epochs}, Loss: {logs['loss']:.6f}, Val Loss: {logs['val_loss']:.6f}",
                      end='\r', flush=True)

    progress_callback = ProgressCallback() if show_progress else None

    train_start_time = datetime.now()

    callbacks_list = [early_stopping, history_callback]
    if progress_callback:
        callbacks_list.append(progress_callback)

    history = model.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=max_epochs,
        batch_size=batch_size,
        callbacks=callbacks_list,
        verbose=0
    )

    train_end_time = datetime.now()
    train_duration = (train_end_time - train_start_time).total_seconds()

    n_epochs = len(history.history['loss'])
    if n_epochs > 0:
        time_per_epoch = train_duration / n_epochs
        epoch_times = [time_per_epoch * (i + 1) for i in range(n_epochs)]
        history.history['epoch_times'] = epoch_times
        history.history['total_time'] = train_duration
    else:
        history.history['epoch_times'] = []
        history.history['total_time'] = 0

    return model, history.history

# ==================== 超參數搜尋（貝葉斯優化） ====================
def hyperparameter_search(X_train, X_val, group_name, indicator_cols):
    """使用貝葉斯優化進行超參數搜尋"""
    print(f"\n{'='*60}")
    print(f"[SEARCH] 開始貝葉斯優化超參數搜尋: {group_name}")
    print(f"   輸入維度: {len(indicator_cols)}")
    print(f"   優化迭代次數: {BAYESIAN_N_CALLS}")

    input_dim = len(indicator_cols)
    print(f"   Bottleneck 固定為: {FIXED_BOTTLENECK}（將 {input_dim} 維壓縮為 1 維）")

    assert len(DROPOUT_RATES) >= 2, f"Dropout 候選必須至少 2 個，當前: {DROPOUT_RATES}"

    dimensions = [
        LEARNING_RATE_SPACE,
        Integer(0, len(DROPOUT_RATES) - 1, name='dropout_idx'),
    ]

    results = []
    best_val_mse = float('inf')
    best_config = None
    best_model = None
    best_history = None
    iteration_count = [0]

    search_start_time = datetime.now()

    @use_named_args(dimensions=dimensions)
    def objective(learning_rate, dropout_idx):
        iteration_count[0] += 1
        idx = iteration_count[0]

        bottleneck = FIXED_BOTTLENECK
        lr = float(learning_rate)
        dropout = DROPOUT_RATES[int(dropout_idx)]
        batch = BATCH_SIZE

        config = {
            'bottleneck': bottleneck,
            'lr': lr,
            'dropout': dropout,
            'batch': batch
        }

        progress = (idx - 1) / BAYESIAN_N_CALLS * 100
        elapsed_time = (datetime.now() - search_start_time).total_seconds()

        print(f"\n  [貝葉斯優化 {progress:.1f}%] [{idx}/{BAYESIAN_N_CALLS}] 測試組合:")
        print(f"    Bottleneck: {bottleneck}, LR: {lr:.0e}, "
              f"Dropout: {dropout}, Batch: {batch}")

        if idx > 1:
            avg_time = elapsed_time / (idx - 1)
            remaining = avg_time * (BAYESIAN_N_CALLS - idx + 1)
            print(f"    已用時間: {elapsed_time:.1f}秒 | 預計剩餘: {remaining:.1f}秒")

        try:
            print("    [訓練中...] ", end='', flush=True)
            model, history = train_autoencoder(
                X_train, X_val,
                bottleneck_size=bottleneck,
                lr=lr,
                dropout_rate=dropout,
                batch_size=batch,
                max_epochs=MAX_EPOCHS,
                patience=SEARCH_EARLY_STOPPING_PATIENCE,
                group_name=group_name
            )

            best_val = float(np.min(history['val_loss']))

            if 'total_time' in history:
                epochs = len(history.get('loss', []))
                print(f"[完成] 訓練時間: {history['total_time']:.2f}秒 ({epochs} epochs)")

            print(f"    [結果] 最佳 Val Loss: {best_val:.6f}")

            nonlocal best_val_mse, best_config, best_model, best_history
            is_better = False
            if best_val < best_val_mse * 0.99:
                is_better = True
            elif best_config is not None and best_val <= best_val_mse * 1.01:
                is_better = True

            train_mse = None
            val_mse = None
            if is_better:
                print("[評估中...] ", end='', flush=True)
                val_pred = model.predict(X_val, verbose=0)
                val_mse = mean_squared_error(X_val, val_pred)

                train_pred = model.predict(X_train, verbose=0)
                train_mse = mean_squared_error(X_train, train_pred)

                print(f"[完成] Train MSE: {train_mse:.6f}, Val MSE: {val_mse:.6f}")

                best_val_mse = best_val
                best_config = config
                best_model = model
                best_history = history
                print("    [BEST] 更新最佳模型！")

            result = {
                'config': config,
                'train_mse': train_mse,
                'val_mse': val_mse,
                'best_val_loss': best_val,
                'model': model if is_better else None,
                'history': history
            }
            results.append(result)

            return best_val

        except Exception as e:
            print(f"    [ERROR] 訓練失敗: {e}")
            return 1e10

    print("\n[INFO] 開始貝葉斯優化（高斯過程）...")
    result_optimization = gp_minimize(
        func=objective,
        dimensions=dimensions,
        n_calls=BAYESIAN_N_CALLS,
        random_state=RANDOM_SEED,
        n_initial_points=min(4, BAYESIAN_N_CALLS),
        acq_func='EI',
        verbose=False
    )

    if best_config is None:
        raise ValueError(f"[ERROR] {group_name} 沒有成功的訓練結果！")

    print(f"\n{'='*60}")
    print(f"[BEST] 最佳超參數 ({group_name}):")
    print(f"   Bottleneck: {FIXED_BOTTLENECK} (固定)")
    print(f"   Learning Rate: {best_config['lr']:.0e}")
    print(f"   Dropout: {best_config['dropout']}")
    print(f"   Batch Size: {best_config['batch']}")
    print(f"   最佳驗證損失: {best_val_mse:.6f}")
    best_result = next((r for r in results if r.get('val_mse') is not None and r['config'] == best_config), None)
    if best_result:
        print(f"   驗證集 MSE: {best_result['val_mse']:.6f}")
    print(f"\n[INFO] 貝葉斯優化找到的最佳目標值: {result_optimization.fun:.6f}")
    print(f"[INFO] 最佳參數位置: {result_optimization.x}")

    return best_model, best_config, best_history, results

# ==================== 最終訓練 ====================
def final_training(X_train, X_val, X_test, best_config, group_name, best_model=None):
    """使用最佳參數在 Train+Val 上重訓，評估 Test"""
    print(f"\n{'='*60}")
    print(f"[FINAL] 最終訓練: {group_name}")
    print("   使用 Train+Val 資料重新訓練...")

    X_train_val = np.vstack([X_train, X_val])

    n_val_final = int(len(X_train_val) * 0.2)
    X_train_final = X_train_val[:-n_val_final]
    X_val_final = X_train_val[-n_val_final:]

    print(f"   最終訓練集: {len(X_train_final):,} 筆")
    print(f"   最終驗證集: {len(X_val_final):,} 筆（用於早停）")
    print(f"   測試集: {len(X_test):,} 筆")

    if best_model is not None:
        print("   [優化] 使用遷移學習：從搜索階段最佳模型繼續訓練（節省時間）")
        input_dim = X_train_final.shape[1]

        model = build_autoencoder(
            input_dim,
            best_config['bottleneck'],
            best_config['dropout']
        )

        optimizer = keras.optimizers.Adam(learning_rate=best_config['lr'])
        model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

        try:
            best_layers = best_model.layers
            new_layers = model.layers

            for best_layer, new_layer in zip(best_layers, new_layers):
                if len(best_layer.get_weights()) > 0 and len(new_layer.get_weights()) > 0:
                    if (best_layer.get_weights()[0].shape == new_layer.get_weights()[0].shape and
                        len(best_layer.get_weights()) == len(new_layer.get_weights())):
                        new_layer.set_weights(best_layer.get_weights())
            print("   ✅ 權重遷移成功")
        except Exception as e:
            print(f"   ⚠️ 權重遷移失敗，將從頭訓練: {e}")

        early_stopping = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=EARLY_STOPPING_PATIENCE,
            restore_best_weights=True,
            verbose=0
        )

        history_callback = callbacks.History()
        train_start_time = datetime.now()

        history = model.fit(
            X_train_final, X_train_final,
            validation_data=(X_val_final, X_val_final),
            epochs=MAX_EPOCHS,
            batch_size=best_config['batch'],
            callbacks=[early_stopping, history_callback],
            verbose=0,
            initial_epoch=0
        )

        train_end_time = datetime.now()
        train_duration = (train_end_time - train_start_time).total_seconds()

        n_epochs = len(history.history['loss'])
        if n_epochs > 0:
            time_per_epoch = train_duration / n_epochs
            epoch_times = [time_per_epoch * (i + 1) for i in range(n_epochs)]
            history.history['epoch_times'] = epoch_times
            history.history['total_time'] = train_duration
        else:
            history.history['epoch_times'] = []
            history.history['total_time'] = 0

        history = history.history
    else:
        model, history = train_autoencoder(
            X_train_final, X_val_final,
            bottleneck_size=best_config['bottleneck'],
            lr=best_config['lr'],
            dropout_rate=best_config['dropout'],
            batch_size=best_config['batch'],
            max_epochs=MAX_EPOCHS,
            patience=EARLY_STOPPING_PATIENCE,
            group_name=group_name
        )

    epochs = len(history.get('loss', []))
    train_time = history.get('total_time', 0)
    print(f"[完成] 訓練時間: {train_time:.2f}秒 ({epochs} epochs)")

    print("   [評估中...] ", end='', flush=True)

    train_val_pred = model.predict(X_train_val, verbose=0)
    test_pred = model.predict(X_test, verbose=0)

    train_val_mse = mean_squared_error(X_train_val, train_val_pred)
    test_mse = mean_squared_error(X_test, test_pred)

    train_pred_only = model.predict(X_train, verbose=0)
    val_pred_only = model.predict(X_val, verbose=0)
    train_mse = mean_squared_error(X_train, train_pred_only)
    val_mse = mean_squared_error(X_val, val_pred_only)

    print("[完成]")
    print(f"   [結果] Train MSE: {train_mse:.6f}")
    print(f"   [結果] Val MSE: {val_mse:.6f}")
    print(f"   [結果] Train+Val MSE: {train_val_mse:.6f}")
    print(f"   [結果] Test MSE: {test_mse:.6f}")

    return model, {
        'train_mse': train_mse,
        'val_mse': val_mse,
        'train_val_mse': train_val_mse,
        'test_mse': test_mse,
        'history': history
    }

# ==================== 壓縮並保存資料 ====================
def compress_and_save_data(model, scaler, df_to_compress, indicator_cols, group_name, output_dir, bottleneck_size):
    """使用訓練好的模型壓縮資料並保存為時間序列格式（只壓縮測試年）"""
    compressed_dir = os.path.join(output_dir, "compressed_data")
    os.makedirs(compressed_dir, exist_ok=True)

    print("   準備壓縮資料...")
    all_data = prepare_indicator_data(df_to_compress, indicator_cols)

    all_data_scaled = scaler.transform(all_data)

    encoder_input = model.input
    encoder_output = None

    for layer in model.layers:
        if layer.name == 'bottleneck':
            encoder_output = layer.output
            break

    if encoder_output is None:
        bottleneck_layer_idx = None
        for i, layer in enumerate(model.layers):
            if layer.name == 'bottleneck':
                bottleneck_layer_idx = i
                break

        if bottleneck_layer_idx is not None:
            encoder_output = model.layers[bottleneck_layer_idx].output
        else:
            raise ValueError("無法找到編碼器輸出層")

    encoder_model = models.Model(inputs=encoder_input, outputs=encoder_output)

    print("   使用編碼器壓縮資料...")
    compressed_data = encoder_model.predict(all_data_scaled, verbose=0)

    compressed_df = pd.DataFrame(
        compressed_data,
        columns=[f"{group_name}_compressed_{i}" for i in range(bottleneck_size)]
    )

    if 'datetime' in df_to_compress.columns:
        compressed_df['datetime'] = df_to_compress['datetime'].values
        cols = ['datetime'] + [col for col in compressed_df.columns if col != 'datetime']
        compressed_df = compressed_df[cols]

    output_path = os.path.join(compressed_dir, f"{group_name}_compressed.csv")
    compressed_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    original_size = all_data.shape[1]
    compressed_size = bottleneck_size
    compression_ratio = original_size / compressed_size

    print("   ✅ 壓縮完成！")
    print(f"   原始維度: {original_size}")
    print(f"   壓縮後維度: {compressed_size}")
    print(f"   壓縮比: {compression_ratio:.2f}:1")
    print(f"   資料筆數: {len(compressed_df):,}")
    print(f"   保存路徑: {output_path}")

    return output_path

# ==================== 繪圖函數 ====================
def plot_training_history(history, group_name, output_path):
    """繪製訓練歷史（包含時間軸）"""
    has_time = 'epoch_times' in history and len(history['epoch_times']) > 0

    if has_time:
        fig, axes = plt.subplots(1, 3, figsize=(20, 5))
        epoch_times = history['epoch_times']
    else:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].plot(history['loss'], label='Train MSE', linewidth=2)
    axes[0].plot(history['val_loss'], label='Val MSE', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('MSE', fontsize=12)
    axes[0].set_title(f'{group_name} - Training MSE (by Epoch)', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)

    axes[1].semilogy(history['loss'], label='Train MSE', linewidth=2)
    axes[1].semilogy(history['val_loss'], label='Val MSE', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('MSE (log scale)', fontsize=12)
    axes[1].set_title(f'{group_name} - Training MSE (Log Scale, by Epoch)', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)

    if has_time:
        axes[2].plot(epoch_times, history['loss'], label='Train MSE', linewidth=2)
        axes[2].plot(epoch_times, history['val_loss'], label='Val MSE', linewidth=2)
        axes[2].set_xlabel('Training Time (seconds)', fontsize=12)
        axes[2].set_ylabel('MSE', fontsize=12)
        axes[2].set_title(f'{group_name} - Training MSE (by Time)\nTotal: {history.get("total_time", 0):.1f}s',
                         fontsize=14, fontweight='bold')
        axes[2].legend(fontsize=10)
        axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    return output_path

# ==================== Excel 輸出 ====================
def save_results_to_excel(all_results, output_dir):
    """將所有結果保存到 Excel"""
    print(f"\n{'='*60}")
    print("[SAVE] 保存結果到 Excel...")

    os.makedirs(output_dir, exist_ok=True)
    excel_path = os.path.join(output_dir, f"autoencoder_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")

    wb = openpyxl.Workbook()

    ws_summary = wb.active
    ws_summary.title = "摘要"
    ws_summary.append(["視窗(訓練+測試)", "技術指標群組", "輸入維度", "最佳 Bottleneck", "最佳 LR", "最佳 Dropout",
                      "最佳 Batch Size", "Train MSE", "Val MSE", "Train+Val MSE", "Test MSE",
                      "訓練時間(秒)", "總訓練時間(秒)"])

    for result in all_results:
        config = result['best_config']
        final_scores = result['final_scores']

        final_history = final_scores.get('history', {})
        final_time = final_history.get('total_time', 0) if 'total_time' in final_history else 0

        search_total_time = 0
        for search_result in result.get('search_results', []):
            if isinstance(search_result, dict) and 'history' in search_result:
                search_total_time += search_result['history'].get('total_time', 0)

        total_training_time = search_total_time + final_time

        ws_summary.append([
            result['window'],
            result['group_name'],
            result['input_dim'],
            config['bottleneck'],
            config['lr'],
            config['dropout'],
            config['batch'],
            f"{final_scores['train_mse']:.6f}",
            f"{final_scores['val_mse']:.6f}",
            f"{final_scores.get('train_val_mse', final_scores['train_mse']):.6f}",
            f"{final_scores['test_mse']:.6f}",
            f"{final_time:.2f}",
            f"{total_training_time:.2f}"
        ])

    existing_sheet_names = set(["摘要"])

    def safe_sheet_name(name, existing_names):
        base = name[:31]
        if base not in existing_names:
            existing_names.add(base)
            return base
        idx = 1
        while True:
            suffix = f"_{idx}"
            trimmed = base[:31 - len(suffix)] + suffix
            if trimmed not in existing_names:
                existing_names.add(trimmed)
                return trimmed
            idx += 1

    for result in all_results:
        window = result['window']
        group_name = result['group_name']
        sheet_name = safe_sheet_name(f"{window}_{group_name}", existing_sheet_names)
        ws = wb.create_sheet(title=sheet_name)

        ws.append(["超參數搜尋結果"])
        ws.append(["Bottleneck", "Learning Rate", "Dropout", "Batch Size", "Train MSE", "Val MSE"])

        for search_result in result['search_results']:
            config = search_result['config']
            ws.append([
                config['bottleneck'],
                config['lr'],
                config['dropout'],
                config['batch'],
                f"{search_result['train_mse']:.6f}",
                f"{search_result['val_mse']:.6f}"
            ])

        ws.append([])
        ws.append(["最佳配置"])
        best_config = result['best_config']
        ws.append(["Bottleneck", best_config['bottleneck']])
        ws.append(["Learning Rate", best_config['lr']])
        ws.append(["Dropout", best_config['dropout']])
        ws.append(["Batch Size", best_config['batch']])

        ws.append([])
        ws.append(["最終評估結果"])
        final_scores = result['final_scores']
        ws.append(["Train MSE", f"{final_scores['train_mse']:.6f}"])
        ws.append(["Val MSE", f"{final_scores['val_mse']:.6f}"])
        if 'train_val_mse' in final_scores:
            ws.append(["Train+Val MSE", f"{final_scores['train_val_mse']:.6f}"])
        ws.append(["Test MSE", f"{final_scores['test_mse']:.6f}"])

        ws.append([])
        ws.append(["訓練時間信息"])
        final_history = final_scores.get('history', {})
        if 'total_time' in final_history:
            ws.append(["最終訓練時間", f"{final_history['total_time']:.2f} 秒"])
            ws.append(["平均每 Epoch 時間", f"{final_history['total_time'] / max(len(final_history.get('loss', [])), 1):.2f} 秒"])
        else:
            ws.append(["最終訓練時間", "未記錄"])

        search_total_time = 0
        for search_result in result.get('search_results', []):
            if 'history' in search_result and 'total_time' in search_result['history']:
                search_total_time += search_result['history']['total_time']
        if search_total_time > 0:
            ws.append(["超參數搜尋總時間", f"{search_total_time:.2f} 秒"])
            ws.append(["總訓練時間", f"{search_total_time + final_history.get('total_time', 0):.2f} 秒"])

        img_path = result['plot_path']
        if os.path.exists(img_path):
            try:
                img = Image(img_path)
                img.width = 800
                img.height = 300
                ws.add_image(img, f'A{ws.max_row + 3}')
            except Exception as e:
                print(f"  [WARN] 無法插入圖片 {img_path}: {e}")

    ws_log = wb.create_sheet(title="訓練日誌")
    ws_log.append(["時間", "視窗", "群組", "階段", "訊息"])

    for result in all_results:
        if 'log' in result:
            for log_entry in result['log']:
                ws_log.append(log_entry)

    wb.save(excel_path)
    print(f"[OK] Excel 已保存: {excel_path}")

    return excel_path

# ==================== 主程式 ====================
def main():
    """主程式"""
    print("=" * 60)
    print("[START] 技術指標 Autoencoder 壓縮訓練")
    print("=" * 60)
    print(f"[TIME] 開始時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"[SEED] 隨機種子: {RANDOM_SEED}")

    print("\n[GPU] GPU 檢查和配置...")
    print(f"TensorFlow 版本: {tf.__version__}")
    print(f"TensorFlow 是否構建時包含 CUDA 支持: {tf.test.is_built_with_cuda()}")

    gpus = tf.config.list_physical_devices('GPU')
    print(f"可用 GPU 清單: {gpus}")

    if len(gpus) > 0:
        print(f"✅ 檢測到 {len(gpus)} 個 GPU 設備")
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("✅ GPU 記憶體增長已啟用")

            logical_gpus = tf.config.list_logical_devices('GPU')
            if len(logical_gpus) > 0:
                print(f"✅ GPU 可用於 TensorFlow 運算: {logical_gpus}")
                try:
                    with tf.device('/GPU:0'):
                        test_tensor = tf.constant([1.0, 2.0, 3.0])
                        result = tf.reduce_sum(test_tensor)
                    print(f"✅ GPU 運算測試成功: {result.numpy()}")
                    print("TensorFlow 是否使用 GPU: True")
                except Exception as e:
                    print(f"⚠️ GPU 運算測試失敗: {e}")
                    print("TensorFlow 是否使用 GPU: False")
            else:
                print("❌ GPU 不可用於 TensorFlow 運算")
                print("TensorFlow 是否使用 GPU: False")
        except RuntimeError as e:
            print(f"⚠️ GPU 設定警告: {e}")
            print("TensorFlow 是否使用 GPU: False")
    else:
        print("❌ 沒有檢測到 GPU 設備")
        if not tf.test.is_built_with_cuda():
            print("⚠️ TensorFlow 當前版本似乎不包含 CUDA 支持（CPU-only 構建）")
            print("💡 提示：如果已安裝 CUDA，可能需要：")
            print("   1. 確保已安裝完整的 CUDA Toolkit（不僅是驅動）")
            print("   2. 安裝對應版本的 cuDNN")
            print("   3. 或考慮使用 conda 安裝支持 GPU 的 TensorFlow")
        print("將使用 CPU 進行運算")
        print("TensorFlow 是否使用 GPU: False")

    print()

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    windows = build_rolling_windows_by_year(df, window_years=3)
    print(f"[INFO] 共找到 {len(windows)} 個 3 年滾動視窗")

    all_results = []
    overall_start_time = datetime.now()

    for window_idx, window_years in enumerate(windows, 1):
        train_years = window_years[:2]
        test_year = window_years[2]
        window_label = f"{train_years[0]}-{test_year}"

        print(f"\n{'='*70}")
        print(f"[WINDOW {window_idx}/{len(windows)}] 3 年視窗: {window_label}")
        print(f"   訓練年: {train_years[0]}, {train_years[1]}")
        print(f"   測試年: {test_year}")
        print(f"{'='*70}")

        window_output_dir = os.path.join(OUTPUT_DIR, f"window_{window_label}")
        models_dir = os.path.join(window_output_dir, "models")
        plots_dir = os.path.join(window_output_dir, "plots")
        os.makedirs(models_dir, exist_ok=True)
        os.makedirs(plots_dir, exist_ok=True)

        train_pool_df = df[df['datetime'].dt.year.isin(train_years)].copy()
        test_df = df[df['datetime'].dt.year == test_year].copy()

        if train_pool_df.empty or test_df.empty:
            print("[WARN] 當前視窗資料不足，跳過。")
            continue

        train_df, val_df = split_train_val_by_date(train_pool_df, VAL_SPLIT)

        total_groups = len(INDICATOR_GROUPS)
        for group_idx, (group_name, indicator_cols) in enumerate(INDICATOR_GROUPS.items(), 1):
            print(f"\n{'#'*60}")
            print(f"[GROUP {group_idx}/{total_groups}] 視窗 {window_label} | 群組: {group_name}")
            print(f"   包含指標: {', '.join(indicator_cols)}")
            print(f"   群組進度: {group_idx}/{total_groups} ({group_idx/total_groups*100:.1f}%)")

            elapsed = (datetime.now() - overall_start_time).total_seconds()
            if group_idx > 1:
                avg_time_per_group = elapsed / (group_idx - 1)
                remaining_groups = total_groups - group_idx + 1
                remaining_time = avg_time_per_group * remaining_groups
                print(f"   已用時間: {elapsed:.1f}秒 | 預計剩餘: {remaining_time:.1f}秒")

            try:
                X_train_raw = prepare_indicator_data(train_df, indicator_cols)
                X_val_raw = prepare_indicator_data(val_df, indicator_cols)
                X_test_raw = prepare_indicator_data(test_df, indicator_cols)

                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train_raw)
                X_val = scaler.transform(X_val_raw)
                X_test = scaler.transform(X_test_raw)

                print("[OK] 資料準備完成")
                print(f"   訓練集形狀: {X_train.shape}")
                print(f"   驗證集形狀: {X_val.shape}")
                print(f"   測試集形狀: {X_test.shape}")

                best_model, best_config, best_history, search_results = hyperparameter_search(
                    X_train, X_val, group_name, indicator_cols
                )

                search_model_path = os.path.join(models_dir, f"{group_name}_search_best.h5")
                best_model.save(search_model_path)

                if SKIP_FINAL_TRAINING:
                    print("\n[OPTIMIZE] 跳過最終訓練，直接使用搜索階段最佳模型")
                    final_model = best_model
                    final_history = best_history

                    print("   [評估中...] ", end='', flush=True)
                    X_train_val = np.vstack([X_train, X_val])

                    train_val_pred = final_model.predict(X_train_val, verbose=0)
                    test_pred = final_model.predict(X_test, verbose=0)

                    train_val_mse = mean_squared_error(X_train_val, train_val_pred)
                    test_mse = mean_squared_error(X_test, test_pred)

                    train_pred_only = final_model.predict(X_train, verbose=0)
                    val_pred_only = final_model.predict(X_val, verbose=0)
                    train_mse = mean_squared_error(X_train, train_pred_only)
                    val_mse = mean_squared_error(X_val, val_pred_only)

                    print("[完成]")
                    print(f"   [結果] Train MSE: {train_mse:.6f}")
                    print(f"   [結果] Val MSE: {val_mse:.6f}")
                    print(f"   [結果] Train+Val MSE: {train_val_mse:.6f}")
                    print(f"   [結果] Test MSE: {test_mse:.6f}")

                    final_scores = {
                        'train_mse': train_mse,
                        'val_mse': val_mse,
                        'train_val_mse': train_val_mse,
                        'test_mse': test_mse,
                        'history': final_history
                    }

                    final_model_path = search_model_path
                else:
                    final_model, final_scores = final_training(
                        X_train, X_val, X_test, best_config, group_name, best_model=best_model
                    )

                    final_model_path = os.path.join(models_dir, f"{group_name}_final.h5")
                    final_model.save(final_model_path)

                plot_path = os.path.join(plots_dir, f"{group_name}_training_history.png")
                plot_training_history(final_scores['history'], group_name, plot_path)

                scaler_path = os.path.join(models_dir, f"{group_name}_scaler.pkl")
                import pickle
                with open(scaler_path, 'wb') as f:
                    pickle.dump(scaler, f)

                print(f"\n[COMPRESS] 壓縮測試年資料: {group_name} | 視窗 {window_label}")
                compressed_data_path = compress_and_save_data(
                    final_model, scaler, test_df, indicator_cols, group_name,
                    window_output_dir, best_config['bottleneck']
                )

                all_results.append({
                    'window': window_label,
                    'group_name': group_name,
                    'input_dim': len(indicator_cols),
                    'best_config': best_config,
                    'search_results': [
                        {
                            'config': r['config'],
                            'train_mse': r['train_mse'],
                            'val_mse': r['val_mse'],
                            'history': r.get('history', {})
                        }
                        for r in search_results
                    ],
                    'final_scores': final_scores,
                    'plot_path': plot_path,
                    'model_path': final_model_path,
                    'scaler_path': scaler_path,
                    'compressed_data_path': compressed_data_path
                })

                print(f"[OK] {group_name} 視窗 {window_label} 處理完成")

            except Exception as e:
                print(f"[ERROR] {group_name} 視窗 {window_label} 處理失敗: {e}")
                import traceback
                traceback.print_exc()
                continue

    excel_path = save_results_to_excel(all_results, OUTPUT_DIR)

    json_path = os.path.join(OUTPUT_DIR, f"results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    json_results = []
    for r in all_results:
        json_results.append({
            'window': r['window'],
            'group_name': r['group_name'],
            'input_dim': r['input_dim'],
            'best_config': r['best_config'],
            'final_scores': r['final_scores'],
            'search_results': r['search_results']
        })

    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_results, f, indent=2, ensure_ascii=False)

    print(f"\n{'='*60}")
    print("[DONE] 所有處理完成！")
    print(f"[DIR] 結果目錄: {OUTPUT_DIR}")
    print(f"[EXCEL] Excel 報告: {excel_path}")
    print(f"[JSON] JSON 報告: {json_path}")

    print(f"[TIME] 結束時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 60)

if __name__ == "__main__":
    main()

[START] 技術指標 Autoencoder 壓縮訓練
[TIME] 開始時間: 2026-02-08 15:56:59
[SEED] 隨機種子: 42

[GPU] GPU 檢查和配置...
TensorFlow 版本: 2.19.0
TensorFlow 是否構建時包含 CUDA 支持: True
可用 GPU 清單: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ 檢測到 1 個 GPU 設備
✅ GPU 記憶體增長已啟用
✅ GPU 可用於 TensorFlow 運算: [LogicalDevice(name='/device:GPU:0', device_type='GPU')]
✅ GPU 運算測試成功: 6.0
TensorFlow 是否使用 GPU: True

[INFO] 改用「交易日」切分資料 (總天數: 2723 天)
  訓練集天數: 1907
  驗證集天數: 544
  測試集天數: 272
  Cutoff 1 (Train|Val): 2020-02-24
  Cutoff 2 (Val|Test) : 2022-09-30
------------------------------------------------------------
  訓練集: 574,007 筆 (70.0%) | 時間: 2011-01-03 08:45:00 至 2020-02-21 13:45:00
  驗證集: 163,744 筆 (20.0%) | 時間: 2020-02-24 08:45:00 至 2022-09-29 13:45:00
  測試集: 81,872 筆 (10.0%) | 時間: 2022-09-30 08:45:00 至 2023-12-22 13:45:00

############################################################
[GROUP 1/7] 處理技術指標群組: STOCH
   包含指標: STOCH_K_14, STOCH_D_14
   總體進度: 1/7 (14.3%)
############################################


[BEST] 最佳超參數 (STOCH):
   Bottleneck: 1 (固定)
   Learning Rate: 8e-04
   Dropout: 0.0
   Batch Size: 32786
   最佳驗證損失: 0.024956
   驗證集 MSE: 0.024955

[INFO] 貝葉斯優化找到的最佳目標值: 0.024949
[INFO] 最佳參數位置: [0.0006989510422011376, np.int64(0)]

[OPTIMIZE] 跳過最終訓練，直接使用搜索階段最佳模型（節省時間）
   [評估中...] [完成]
   [結果] Train MSE: 0.024371
   [結果] Val MSE: 0.024955
   [結果] Train+Val MSE: 0.024500
   [結果] Test MSE: 0.026518

[COMPRESS] 開始壓縮時間序列資料: STOCH
   準備壓縮資料...
   使用編碼器壓縮資料...
   ✅ 壓縮完成！
   原始維度: 2
   壓縮後維度: 1
   壓縮比: 2.00:1
   資料筆數: 819,623
   保存路徑: /content/drive/MyDrive/工作區/論文/論文code/1106/output2/compressed_data/STOCH_compressed.csv
[OK] STOCH 處理完成！ (耗時: 719.8秒)
   已完成 1/7 個群組 (14.3%)

############################################################
[GROUP 2/7] 處理技術指標群組: STOCHF
   包含指標: STOCHF_K_14, STOCHF_D_14
   總體進度: 2/7 (28.6%)
   已用時間: 719.8秒 | 預計剩餘: 4318.7秒 (72.0分鐘)
############################################################
[OK] 資料準備完成
   訓練集形狀: (574007, 2)
   驗證集形狀: (163744, 2)
   測試集形狀: (81872, 2)

[